In [1]:
import numpy as np
import pandas as pd
import os
import torch
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torch.nn import Linear,Softmax,CrossEntropyLoss,Module,ReLU,DataParallel,Sequential
from torch.optim import Adam, SGD
from tqdm import tqdm_notebook
from nltk import word_tokenize
from pymagnitude import *
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm
from time import time
from sentence_transformers import SentenceTransformer
import re

import warnings
warnings.filterwarnings('ignore')

device = 'cuda:0'

/home/nikamanth/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/nikamanth/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/nikamanth/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/nikamanth/.local/lib/python3.7/site-packages/tensorflow/python/framework/dty

In [2]:
train_df = pd.read_csv('../tsv/oversample_train.tsv','\t')
dev_df = pd.read_csv('../NADI-2020_release_1.0/NADI_release/dev_labeled.tsv',sep='\t')
X_train_corrected_tweets = []
for tweet in tqdm(train_df['#2 tweet_content']):
    new_tweet = re.findall( '[^A-Za-z:/_.0-9\\#@,=+\(\)]+' ,tweet)
    new_tweet = " ".join(new_tweet).replace('\xa0','').replace('\u200c','').replace('\U000fe329','').replace('\u2066','').replace('\u2069','').strip()
    X_train_corrected_tweets.append(new_tweet)
    
X_dev_corrected_tweets = []
for tweet in tqdm(dev_df['#2 tweet_content']):
    new_tweet = re.findall( '[^A-Za-z:/_.0-9\\#@,=+\(\)]+' ,tweet) #[^\x00-\x19\x21-\x7F]+
    new_tweet = " ".join(new_tweet).replace('\xa0','').replace('\u200c','').replace('\U000fe329','').replace('\u2066','').replace('\u2069','').strip()
    X_dev_corrected_tweets.append(new_tweet)

100%|██████████| 4957/4957 [00:00<00:00, 281254.34it/s]


In [3]:
train_df['#2 tweet_content'] = X_train_corrected_tweets
dev_df['#2 tweet_content'] = X_dev_corrected_tweets

In [4]:
label_map = {}
y_train_original = train_df["#3 country_label"]
for u in range(len(y_train_original.unique())):
    label_map[y_train_original.unique()[u]] = int(u)
reverse_label_map = {value : key for (key, value) in label_map.items()}
def label_onehot(label):
    onehot = np.zeros((21))
    index = label_map[label]
    onehot[index] = 1
    return onehot

In [5]:
class ArabicDataset(Dataset):
    def __init__(self, csv_file=None, million_csv=None, transform=None):
        fast = Magnitude("../downloads/fasttext-arabic/fasttext-arabic.magnitude")
        def transform(x):
            vectors = []
            for title in tqdm(x):
                vectors.append(np.average(fast.query(word_tokenize(str(title))), axis = 0))
            return np.array(vectors)
        bert = SentenceTransformer('distiluse-base-multilingual-cased',device=device)
        def berttext(x):
            return bert.encode(x,show_progress_bar=False,
                                          batch_size=1000)
        
        self.csv_file = csv_file
        if csv_file:         
            if million_csv:
                self.text_df = pd.concat([pd.read_csv(csv_file, sep='\t'),
                                         pd.read_csv(million_csv, sep='\t')])
            else:
                self.text_df = pd.read_csv(csv_file,sep='\t')
        else:
            self.text_df = pd.read_csv(million_csv,sep='\t')
#         self.berttext_data = np.array(berttext(self.text_df['#2 tweet_content']))
        self.fasttext_data = transform(self.text_df['#2 tweet_content'])
#         self.combined_data = np.hstack([self.fasttext_data, self.berttext_data])
    def __len__(self):
        return len(self.text_df)
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        text = self.fasttext_data[idx]
#         text = fasttext([text]).reshape((300))
        if self.csv_file:
            label = self.text_df.iloc[idx]['#3 country_label']
            label = label_onehot(label)
            sample = {'text': text, 
                      'label': torch.from_numpy(label).to(device)}
        else:
            sample = {'text':text}
        return sample

In [6]:
class TuningNet(Module):
    def __init__(self, D_in, H,D_out):
        super(TuningNet, self).__init__()
        self.conv1 =  torch.nn.Conv1d(1, 512, 1)
        self.flatten = torch.nn.Flatten()
        self.linear1 = torch.nn.Linear(D_in, H)
        self.relu = ReLU()
#         self.linear1_1 = torch.nn.Linear(H, H)
#         self.relu2 = ReLU()
        self.linear2 = torch.nn.Linear(H, D_out)
        self.softmax = Softmax(dim=1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.linear1(x)
        x = self.relu(x)
#         x = self.linear1_1(x)
#         x = self.relu2(x)
        x = self.linear2(x)
        y_pred = self.softmax(x)
        return y_pred

In [7]:
train_csv_path = '../NADI-2020_release_1.0/NADI_release/train_labeled.tsv'
dev_csv_path = '../NADI-2020_release_1.0/NADI_release/dev_labeled.tsv'
num_of_epochs = 50
learning_rate = 0.001
model = TuningNet(300,512,21).to(device)
train_batch_size = 32

In [8]:
# learning_rate=0.01

In [9]:
traindataset = ArabicDataset(train_csv_path) #,million_csv_path)

  1%|          | 167/21000 [00:29<1:00:40,  5.72it/s]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/nikamanth/anaconda3/envs/torch/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-9-3164553ceee0>", line 1, in <module>
    traindataset = ArabicDataset(train_csv_path) #,million_csv_path)
  File "<ipython-input-5-dd29a1012e5d>", line 24, in __init__
    self.fasttext_data = transform(self.text_df['#2 tweet_content'])
  File "<ipython-input-5-dd29a1012e5d>", line 7, in transform
    vectors.append(np.average(fast.query(word_tokenize(str(title))), axis = 0))
  File "/home/nikamanth/anaconda3/envs/torch/lib/python3.7/site-packages/pymagnitude/third_party/repoze/lru/__init__.py", line 390, in cached_wrapper
    val = func(*args, **kwargs)
  File "/home/nikamanth/anaconda3/envs/torch/lib/python3.7/site-packages/pymagnitude/__init__.py", line 1221, in query
    vectors = self._vectors_for_keys_cached(q, normalized)
  File "/home/nik

KeyboardInterrupt: 

In [ ]:
trainloader = DataLoader(traindataset, batch_size=32,
                        shuffle=True)
# testloader = DataLoader(traindataset, batch_size=1000)
devdataset = ArabicDataset(dev_csv_path)
devloader = DataLoader(devdataset, batch_size=1000)

In [32]:
criterian = CrossEntropyLoss().to(device)
# optimizer = SGD(model.parameters(), lr=learning_rate,momentum=0.9,nesterov=True)
optimizer = Adam(model.parameters(), lr=learning_rate)
train_f1 = []
dev_f1 = []
for epoch in range(num_of_epochs):
    i_batch = 0
    y_pred = []
    y_true = []
    print("epoch:",epoch)
    for batch in tqdm(trainloader):
        i_batch +=1
        output = model(batch['text'].to(device))
        loss = criterian(output,batch['label'].argmax(dim=1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        y_pred += list(output.argmax(dim=1).detach().cpu().numpy())
        y_true += list(batch['label'].argmax(dim=1).detach().cpu().numpy()) 
    print("training acc:",accuracy_score(y_pred,y_true),end=' ')
    f1 = f1_score(y_pred,y_true,average='macro')
    train_f1.append(f1)
    print("training f1_score:", f1)
    
    
    y_pred = []
    y_true = []
    for batch in devloader:
        output = model(batch['text'].to(device))
        y_pred += list(output.argmax(dim=1).detach().cpu().numpy())
        y_true += list(batch['label'].argmax(dim=1).detach().cpu().numpy())
    print("dev acc:",accuracy_score(y_pred,y_true),end=' ')
    f1 = f1_score(y_pred,y_true,average='macro')
    dev_f1.append(f1)
    print("training f1_score:", f1)

  0%|          | 0/155 [00:00<?, ?it/s]

epoch: 0


AttributeError: 'list' object has no attribute 'to'

In [33]:
# torch.save(model,'../semi-supervised_train/models/task1-sgd-lvl2-2000-improve.pt')

In [25]:
# model = torch.load('../semi-supervised_train/models/fastest.pt').to(device)